<a href="https://colab.research.google.com/github/tleyden/FruitPunch_AI_Bootcamp/blob/main/FruitPunch_Capstone_Project_YOLO_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pip installs

In [ ]:
!apt install -y jq
!pip install pycocotools
!python -m pip install pyyaml==5.1
!pip install wandb -qU


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libjq1 libonig4
The following NEW packages will be installed:
  jq libjq1 libonig4
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 276 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libonig4 amd64 6.7.0-1 [119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libjq1 amd64 1.5+dfsg-2 [111 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 jq amd64 1.5+dfsg-2 [45.6 kB]
Fetched 276 kB in 0s (829 kB/s)
Selecting previously unselected package libonig4:amd64.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../libo

## Imports

In [28]:
import os
from matplotlib import pyplot as plt
import cv2

from pycocotools.coco import COCO
import skimage.io as io
from pathlib import Path

import sys, distutils.core
from google.colab.patches import cv2_imshow
import json
import shutil

## Mount google drive

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount("/content/drive")
     

Mounted at /content/drive


## Download zips from google bucket

In [ ]:
from google.colab import auth
auth.authenticate_user()

if not os.path.exists("Labeled data-20211126T095740Z-001.zip"):
    !gsutil cp "gs://fruitpunch-ai-tleyden/Labeled data-20211126T095740Z-001.zip" .
if not os.path.exists("Labeled data-20211126T095740Z-002.zip"):
    !gsutil cp "gs://fruitpunch-ai-tleyden/Labeled data-20211126T095740Z-002.zip" .


Copying gs://fruitpunch-ai-tleyden/Labeled data-20211126T095740Z-001.zip...
| [1 files][  2.0 GiB/  2.0 GiB]   79.9 MiB/s                                   
Operation completed over 1 objects/2.0 GiB.                                      
Copying gs://fruitpunch-ai-tleyden/Labeled data-20211126T095740Z-002.zip...
/ [1 files][809.6 MiB/809.6 MiB]   83.7 MiB/s                                   
Operation completed over 1 objects/809.6 MiB.                                    


In [ ]:
if not os.path.exists("labeled_data"):
    !unzip -q "Labeled data-20211126T095740Z-001.zip"
    !unzip -q "Labeled data-20211126T095740Z-002.zip"

## Re-arrange directories to match expected structure

In [20]:
if not os.path.exists("labeled_data"):
    os.makedirs("labeled_data")
    !mv "Labeled data" labeled_data
if not os.path.exists("labeled_data/images/test"):
    !mkdir labeled_data/images/test 
    !mv labeled_data/images/*.PNG labeled_data/images/test 

Already have labeled_data dir


## Set some global variables

In [23]:
DATA_PATH = '/content/labeled_data/'


use_single_batch = True
if use_single_batch:
    # Smaller subset of dataset for faster iteration
    LABELS_PATH = DATA_PATH + 'annotations_single_batch/'
    IMAGES_PATH = DATA_PATH + 'images_single_batch/'
else:
    LABELS_PATH = DATA_PATH + 'annotations/'
    IMAGES_PATH = DATA_PATH + 'images/'

# Get paths to IMAGE directories
TRAIN_IMAGES_PATH = IMAGES_PATH + 'train/'
TEST_IMAGES_PATH = IMAGES_PATH + 'test/'
VAL_IMAGES_PATH = IMAGES_PATH + 'val/'

TRAIN_LABELS = LABELS_PATH + 'instances_train.json'
TEST_LABELS = LABELS_PATH + 'instances_test_dataset.json'
VAL_LABELS = LABELS_PATH + 'instances_val.json'



## Create single batch annotations

Create a smaller dataset that can be used for faster iteration

In [24]:
def create_single_batch_annotations(num_instances_to_collect):
    """
    Create a copy of annotations/instances_train.json, instances_test_dataset.json and instances_val.json
    in the annotations_single_batch directory with a smaller subset of annotations.
    """
    source_annotations = ["instances_train.json", "instances_test_dataset.json", "instances_val.json"]
    for source_annotation in source_annotations:
        subset_json = {}
        with open(os.path.join(DATA_PATH, "annotations", source_annotation), "r") as f:
            instances_json = json.loads(f.read())
        subset_json["licenses"] = instances_json["licenses"]
        subset_json["info"] = instances_json["info"]
        subset_json["categories"] = instances_json["categories"]
        subset_images = []
        subset_annotations = []
        collected_image_ids = []
        
        for annotation_json in instances_json["annotations"]:
            if len(subset_annotations) >= num_instances_to_collect:
                # We have collected enough
                break
            subset_annotations.append(annotation_json)
            collected_image_ids.append(annotation_json["image_id"])
            
        for image_json in instances_json["images"]:
            if image_json["id"] not in collected_image_ids:
                continue
            subset_images.append(image_json)

            # Copy image to target image dir
            if "test" in source_annotation:
                target_img_dir = "test"
            elif "train" in source_annotation:
                target_img_dir = "train"
            elif "val" in source_annotation:
                target_img_dir = "val"

            file_name = image_json["file_name"]
            source_img = os.path.join(DATA_PATH, "images", target_img_dir, file_name)
            target_img = os.path.join(DATA_PATH, "images_single_batch", target_img_dir, file_name)
            if not os.path.exists(target_img):
                os.link(source_img, target_img)


        print(f"For {source_annotation} collected {len(subset_annotations)} instances")
        subset_json["images"] = subset_images
        subset_json["annotations"] = subset_annotations

        # Write json to target dir
        source_annotation_full_path = os.path.join(DATA_PATH, "annotations_single_batch", source_annotation)
        with open(source_annotation_full_path, "w") as f:
            json.dump(subset_json, f)
        print(f"Wrote {source_annotation_full_path}")




In [25]:
if use_single_batch:
    if not os.path.exists(os.path.join(DATA_PATH, "annotations_single_batch")):
        os.makedirs(os.path.join(DATA_PATH, "annotations_single_batch"))
    if not os.path.exists(os.path.join(DATA_PATH, "images_single_batch")):
        os.makedirs(os.path.join(DATA_PATH, "images_single_batch"))
        os.makedirs(os.path.join(DATA_PATH, "images_single_batch", "train"))
        os.makedirs(os.path.join(DATA_PATH, "images_single_batch", "test"))
        os.makedirs(os.path.join(DATA_PATH, "images_single_batch", "val"))

create_single_batch_annotations(64)

For instances_train.json collected 64 instances
Wrote /content/labeled_data/annotations_single_batch/instances_train.json
For instances_test_dataset.json collected 64 instances
Wrote /content/labeled_data/annotations_single_batch/instances_test_dataset.json
For instances_val.json collected 64 instances
Wrote /content/labeled_data/annotations_single_batch/instances_val.json


## Convert annotations from coco -> yolo format

In [ ]:
!git clone https://github.com/ultralytics/JSON2YOLO.git  # clone repo
%pip install -qr JSON2YOLO/requirements.txt # install dependencies

Cloning into 'JSON2YOLO'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 277 (delta 102), reused 84 (delta 82), pack-reused 164
Receiving objects: 100% (277/277), 74.65 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (177/177), done.
/content/JSON2YOLO


In [26]:
from general_json2yolo import convert_coco_json

convert_coco_json('/content/labeled_data/annotations_single_batch')


Annotations /content/labeled_data/annotations_single_batch/instances_test_dataset.json: 100%|██████████| 12/12 [00:00<00:00, 2732.74it/s]
Annotations /content/labeled_data/annotations_single_batch/instances_train.json: 100%|██████████| 64/64 [00:00<00:00, 8125.79it/s]
Annotations /content/labeled_data/annotations_single_batch/instances_val.json: 100%|██████████| 64/64 [00:00<00:00, 7484.61it/s]


## Copy images to yolo dir

In [36]:
if use_single_batch:
    imgs_source_dir = os.path.join("labeled_data", "images_single_batch")
else:
    imgs_source_dir = os.path.join("labeled_data", "images")

imgs_train = os.path.join(imgs_source_dir, "train")
imgs_test = os.path.join(imgs_source_dir, "test")
imgs_val = os.path.join(imgs_source_dir, "val")

imgs_target_dir = os.path.join("new_dir", "images")
imgs_target_train =  os.path.join(imgs_target_dir, "train")
imgs_target_test =  os.path.join(imgs_target_dir, "test")
imgs_target_val =  os.path.join(imgs_target_dir, "val")

if os.path.exists(imgs_target_train):
    shutil.rmtree(imgs_target_train)
if os.path.exists(imgs_target_test):
    shutil.rmtree(imgs_target_test)
if os.path.exists(imgs_target_val):
    shutil.rmtree(imgs_target_val)

shutil.copytree(imgs_train, imgs_target_train)
shutil.copytree(imgs_test, imgs_target_test)
shutil.copytree(imgs_val, imgs_target_val)


'new_dir/images/val'

## Create yolo data yaml file

In [ ]:
yolo_data_yaml = {
    "nc": 1
}